#1: Data Preprocessing and Model Training

##1.1: Import Libraries and Load Data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Load the dataset
file_path = "/content/ObesityDataSet_raw_and_data_sinthetic.csv"
data = pd.read_csv(file_path)

# Preview the first few rows
data.head()


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


##1.2: Data Preprocessing

In [ ]:
# Encode categorical variables
label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Feature scaling
scaler = StandardScaler()
numerical_cols = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Split data into features (X) and target (y)
X = data.drop('NObeyesdad', axis=1)
y = data['NObeyesdad']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


##1.3: Train a Random Forest Classifier

In [ ]:
# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.98      0.98        56
           1       0.90      0.90      0.90        62
           2       0.99      0.97      0.98        78
           3       0.97      0.98      0.97        58
           4       1.00      1.00      1.00        63
           5       0.88      0.89      0.88        56
           6       0.98      0.96      0.97        50

    accuracy                           0.96       423
   macro avg       0.96      0.96      0.96       423
weighted avg       0.96      0.96      0.96       423



##1.4: Save the Trained Model

In [ ]:
# Save the model and preprocessing objects
joblib.dump(model, 'obesity_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')


['label_encoders.pkl']

#Set Up Flask in Google Colab

In [ ]:
!pip install Flask pyngrok joblib

##Import Required Libraries:

In [ ]:
import joblib
from flask import Flask, request, jsonify
import numpy as np

#3: Create the Flask API

In [ ]:
# Load the model and preprocessing objects
model = joblib.load('obesity_model.pkl')
scaler = joblib.load('scaler.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Initialize Flask app
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    # Get JSON data from request
    data = request.json

    # Extract and preprocess input features
    input_data = [
        data['Gender'], data['Age'], data['Height'], data['Weight'],
        data['family_history_with_overweight'], data['FAVC'],
        data['FCVC'], data['NCP'], data['CAEC'], data['SMOKE'],
        data['CH2O'], data['SCC'], data['FAF'], data['TUE'],
        data['CALC'], data['MTRANS']
    ]

    # Encode categorical variables
    for i, col in enumerate(label_encoders.keys()):
        input_data[i] = label_encoders[col].transform([input_data[i]])[0]

    # Standardize numerical variables
    numerical_indices = [1, 2, 3, 6, 7, 10, 12, 13]
    input_data = np.array(input_data).reshape(1, -1)
    input_data[:, numerical_indices] = scaler.transform(input_data[:, numerical_indices])

    # Make prediction
    prediction = model.predict(input_data)[0]

    # Return the result as JSON
    return jsonify({'prediction': int(prediction)})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


#4: Run Flask and Expose via ngrok

##Launch Flask in the Background

In [ ]:
from flask import Flask
import threading

# Run Flask app in a separate thread to prevent blocking Colab
def run_app():
    app.run(port=5000)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


##Expose Flask App Using ngrok:

In [ ]:
from pyngrok import ngrok

# Connect ngrok to localhost:5000 where Flask is running
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")


ERROR:pyngrok.process.ngrok:t=2024-10-27T08:58:00+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-10-27T08:58:00+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-10-27T08:58:00+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

#5. Test the API

In [ ]:
import requests

# Sample input data
input_data = {
    "Gender": "Male",
    "Age": 25,
    "Height": 1.75,
    "Weight": 85,
    "family_history_with_overweight": "yes",
    "FAVC": "yes",
    "FCVC": 3,
    "NCP": 3,
    "CAEC": "Sometimes",
    "SMOKE": "no",
    "CH2O": 2,
    "SCC": "no",
    "FAF": 2,
    "TUE": 1,
    "CALC": "Sometimes",
    "MTRANS": "Public_Transportation"
}

# Make the POST request
response = requests.post(f"{public_url}/predict", json=input_data)

# Print the response from the API
print(f"Prediction: {response.json()}")


NameError: name 'public_url' is not defined